### 3.6
#### 4.) Use the method of Example 3 to find integers x and y such that $x^2+y^2=89753$, given that this number is prime.

In [1]:
import numpy as np
from copy import deepcopy
from math import ceil, floor

def bqf_app(trans, mat):
    return np.matmul(np.matmul(np.transpose(trans), mat), trans)

class Bqf:
    def __init__(self, a, b, c):
        self.terms = [["x^2", a, lambda x, y: x ** 2],
                      ["xy", b, lambda x, y: x * y],
                      ["y^2", c, lambda x, y: y]]
        
        self.update_vals(a,b,c)
        
        self.mat = [
            [self.a, self.b / 2],
            [self.b / 2, c]
        ]
        
        self.det  = self.determinant()
        self.disc = self.discriminant()
        
        if (self.disc == 0): self.type = "Perfect square"
            
        elif (self.disc < 0):
            if (self.a > 0): self.type = "Positive definite"
            else:            self.type = "Negative definite"
                
        else:                self.type = "Indefinite"
        
        self.transform = []
        
    def __call__(self, x, y):
        return sum([i[1] * i[2](x, y) for i in self.terms])
    
    def __repr__(self):
        out = ""
        for i in range(len(self.terms)):
            if self.terms[i][1] != 0:
                
                if i > 0:
                    if self.terms[i][1] > 0:
                        out += "+ "
                    else:
                        out += "- "
                elif (self.terms[i][1] < 0):
                    out += "-"
                        
                if abs(self.terms[i][1]) != 1:
                    out += str(abs(self.terms[i][1]))
                    
                out += self.terms[i][0] + " "

        return out[:-1]
    
    def __eq__(self, other):
        return np.allclose(self.reduce().mat, other.reduce().mat)
    
    def determinant(self):
        return (self.mat[0][0] * self.mat[1][1]) - (self.mat[0][1] * self.mat[1][0])
    
    def discriminant(self):
        return self.determinant() * -4    
    
    def is_reduced(self):
        return -abs(self.a) < self.b <= abs(self.a) < abs(self.c) or\
                0 <= self.b <= abs(self.a) == abs(self.c)
    
    def update_vals(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
        self.terms[0][1] = a
        self.terms[1][1] = b
        self.terms[2][1] = c
        
    def reduce(self):
        if (self.is_reduced()):
            print("already reduced")
            return self
        
        red = deepcopy(self)
        trans = [1,0]
        
        while (not red.is_reduced()):
            if abs(red.c) < abs(red.a) or (abs(red.c) == abs(red.a) and -abs(red.a) <= red.b < 0):
                red.mat = bqf_app([[0, 1],[-1, 0]], red.mat)
                trans = np.linalg.inv([[0, 1],[-1, 0]]) @ trans
            else:
                m = abs(floor(red.b / red.a)) * (1 if red.a > red.b else -1) / 2
                red.mat = bqf_app([[1, m],[0, 1]], red.mat)
                trans = np.linalg.inv([[1, m],[0, 1]]) @ trans
            
            red.update_vals(red.mat[0][0], red.mat[0][1] * 2, red.mat[1][1])
        return red, trans

In [2]:
prime_list = set([2] + [*filter(lambda i:all(i%j for j in range(3,i,2)), range(3,100000,2))])

In [3]:
# find two integers such that a^2 + b^2 = prime = 1 mod 4
def find_xy(p):
    # make sure number is good
    assert p in prime_list
    assert p % 4 == 1
    # find a starting equation
    c = 1
    while (int((4 * p * c - 4) ** .5) != (4 * p * c - 4) ** .5): c += 1
    b = Bqf(p, (4 * p * c - 4) ** .5, c)
    # reduce, verify, and return
    reduced = b.reduce()[1]
    if reduced[0] ** 2 + reduced[1] ** 2 != p:
        print(p, reduced)
        assert 0
    return int(reduced[0]), int(reduced[1])

In [4]:
x, y = find_xy(89753)
print("({})^2 + ({})^2 = {}".format(x, y, x ** 2 + y ** 2))

(-67)^2 + (292)^2 = 89753


I was trying to find all of the values of a and b possible to generate a prime, but weirdly, my program broke at 13. Something to do with the way I'm handling fractions, I believe. Regardless, the program can't be too bad, as it handles the case for 89753...

### 9.1

#### 1.) If $f(x)|g(x)$ and $g(x)|f(x)$, prove that there is a rational number c such that $g(x) = cf(x)$. 

$f(x)|g(x) \implies \exists u_1(x): f(x)u_1(x)=g(x)$.

Likewise, $g(x)|f(x) \implies \exists u_2(x): g(x)u_2(x)=f(x)$.

Using substitution, we can form the following equation:

$f(x)=f(x)u_1(x)u_2(x) \implies u_1(x)u_2(x)=1 \implies u_1(x)=u_2(x)^{-1}$

Note that if $u_2(x)$ was not a rational number, its inverse would not be in $\mathbb{Q}[x]$, as one of its terms would be of the form $x^{-i}$ for some positive integer i.

Therefore, $u_2(x)$ must be a rational number, which implies that $u_1(x) = u_2(x)^{-1}$ is a rational number.

If we set $c = u_1(x)$, $g(x)=cf(x)$.

#### 2.) If $f(x)|g(x)$ and $g(x)|h(x)$, prove that $f(x)|h(x)$.

$f(x)|g(x) \implies \exists u_1(x): f(x)u_1(x)=g(x)$.

$g(x)|h(x) \implies \exists u_2(x): g(x)u_2(x)=h(x)$.

$\implies f(x)u_1(x)u_2(x)=h(x)$ by substitution.

Let $u_3(x)=u_1(x)u_2(x)$, so $f(x)u_3(x)=h(x)$.

Therefore, $f(x)|h(x)$.

#### 3.) If $p(x)$ is irreducible and $g(x)|p(x)$, prove that either $g(x)$ is a constant or $g(x) = cp(x)$ for some rational number $c$. 

Because $p(x)$ is irreducable, it cannot be factored.

However, observe that any polynomial in $\mathbb{Q}[x]$ can be divided by some constant in $\mathbb{Q}$ without leaving $\mathbb{Q}[x]$.

Additionally, if $g(x)$ is some constant multiple of $p(x)$, $\frac{p(x)}{g(x)}=c$ where $c$ is some rational number.

Any other case, however, would imply that $p(x)$ is a nontrivial polynomial, so the above two cases are the only possible for $g(x)|p(x)$.

#### 5.) If a polynomial $f(x)$ with integral coefficients factors into a product $g(x)h(x)$ of two polynomials with coefficients in $\mathbb{Q}$, prove that there is a factoring $g_1(x)h_1(x)$ with integral coefficients. 

I got help from you on this one.

Tools we need:

1. If $f(x)$ is any rational polynomial, $\exists r: rf(x)$ is a primitive polynomial. This can be found by finding a constant which generates an equivalent integral polynomial and multiplying that constant with the gcd of the integral polynomial.

2. The product of two primitive polynomials are primitive.

3. If $f(x)$ is primitive, $cf(x)$ is primitive $\iff c=\pm 1$

Case 1: $f(x)$ is a primitive polynomial.

First, we find $a,b: ag(x)$ and $bh(x)$ are primitive. We know that $(ab)(g(x)h(x))$ is primitive by (2). By (3), $f(x)$ and $abf(x)$ being primitive imply that $ab=\pm 1$. $g_1(x)=ag(x)$ and $h_1(x)=bh(x)$ fit our requirements.

Case 2: $f(x)$ is any polynomial with integer coefficients.

From (1), we can find a primitive polynomial from $f(x)$ by dividing it by the gcd of its coefficients which we will represent by the constant $c$. $f_1(x)=\frac{1}{c}f(x)\implies \frac{1}{c}g(x)h(x)=f_1(x)$. From the previous case, we know that $\exists a,b:\frac{1}{c}ag(x)bh(x)=f_1(x)$ where $g(x), h(x)$ are integral polynomials. We also know from the previous case that $ab=\pm 1$. $g_1(x)=ag(x)$ and $h_1(x)=bh(x)$ now fit our requirements.

#### 6.)  If $f(x)$ and $g(x)$ are primitive polynomials, and if $f(x)|g(x)$ and $g(x)|f(x)$, prove that $f(x) = \pm g(x)$. 

As proved in problem 9.1.1, $\exists c: g(x) = c f(x)$.

Let us assume that $c$ is not $\pm 1$.

Because $c|g(x)$, we've found a contradiction to $f(x)$'s primitivity, as a primitive polynomial must also be irreducable, but $g(x)$ is a nontrivial polynomial which divides $f(x)$. 

Therefore, $c$ must be $\pm 1$. Again, this case works as an irreducable can only be divided by a unit multiple of itself or a unit.